In [1]:
import os

from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

SCOPES = ['https://www.googleapis.com/auth/calendar']

if os.path.exists('./res/token.json'):
    creds = Credentials.from_authorized_user_file('./res/token.json', SCOPES)
else:
    flow = InstalledAppFlow.from_client_secrets_file(
        './res/credentials.json',
        SCOPES
    )

    creds = flow.run_local_server(port=0)

    with open('./res/token.json', 'w') as token:
        token.write(creds.to_json())

service = build('calendar', 'v3', credentials=creds)




In [2]:
import anthropic

client = anthropic.Anthropic()

response = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    messages=[{"role": "user", "content": "What's the weather in like in San Francisco?"}],
)


In [3]:
response.content

[TextBlock(text="I apologize, but I don't have access to real-time weather information. Weather conditions can change rapidly, and without current data, I can't provide an accurate report for San Francisco's weather at this moment.\n\nHowever, I can tell you that San Francisco generally has a Mediterranean climate with mild temperatures year-round. The city experiences:\n\n1. Dry, warm summers (June to October)\n2. Cool, wet winters (November to May)\n3. Frequent fog, especially during summer mornings\n4. Average temperatures ranging from about 50°F (10°C) to 70°F (21°C) throughout the year\n\nFor current weather conditions and forecasts, I recommend checking a reliable weather website or app that provides up-to-date information for San Francisco.", type='text')]

In [4]:
response = client.messages.create(
  model="claude-3-5-sonnet-20240620",
  max_tokens=1024,
  tools=[
    {
      "name": "get_weather",
      "description": "Get the current weather in a given location",
      "input_schema": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA"
          }
        },
        "required": ["location"]
      }
    }
  ],
  messages=[{'role': 'user', 'content': 'What is the weather in San Francisco?'}]
)

In [5]:
response.content

[TextBlock(text="Certainly! I can help you get the current weather information for San Francisco. To do this, I'll use the weather tool available to me. Let me fetch that information for you right away.", type='text'),
 ToolUseBlock(id='toolu_01JobNpLJVPhDNUUoynTUmZ4', input={'location': 'San Francisco, CA'}, name='get_weather', type='tool_use')]

# Function Call + Google Calendar API

In [7]:
tools = [
  {
    "name": "create_event",
    "description": "Create new Google Calendar event",
    "input_schema": {
      "type": "object",
      "properties": {
        "summary": {
          "type": "string",
          "description": "Name of Google Calendar event"
        },
        "start": {
          "type": "string",
          "description": "Start date of Google Calendar Event in UTC+9 Time i.e. 2024-08-14T09:00:00"
        },
        "end": {
          "type": "string",
          "description": "End date of Google Calendar Event in UTC+9 Time i.e. 2024-08-14T18:00:00"
        },
               
      },
      "required": ["summary"]
    }
  }
]

In [8]:
def create_event(summary, start, end):
  event = {
    "summary": summary,
    "start": {
      "dateTime": start,
      "timeZone": "Asia/Seoul"
    },
    "end": {
      "dateTime": end,
      "timeZone": "Asia/Seoul"
    }
  }

  event = service.events().insert(calendarId='primary', body=event).execute()
  print(f"Event created: %s", (event.get('htmlLink')))
  return event

In [17]:
response = client.messages.create(
  model="claude-3-5-sonnet-20240620",
  max_tokens=1024,
  tools=tools,
  messages=[{'role': 'user', 'content': 'Create an event called Lunch starting from 2024.08.14 1pm'}],
)

In [18]:
response.content

[TextBlock(text='Certainly! I can help you create a Google Calendar event for your lunch using the available tool. Let me set that up for you using the information you provided.', type='text'),
 ToolUseBlock(id='toolu_01B4zB5RgxU3ey33eq5hhwM7', input={'summary': 'Lunch', 'start': '2024-08-14T13:00:00', 'end': '2024-08-14T14:00:00'}, name='create_event', type='tool_use')]

In [19]:
response.content[1].input

{'summary': 'Lunch',
 'start': '2024-08-14T13:00:00',
 'end': '2024-08-14T14:00:00'}

In [20]:
summary = response.content[1].input['summary']
start = response.content[1].input['start']
end = response.content[1].input['end']
create_event(summary, start, end)

Event created: %s https://www.google.com/calendar/event?eid=YW92NTZtMzdyYmI2cnJkcDFncTFkNW4ycmsgbmV2ZXJsaXNoQG0


{'kind': 'calendar#event',
 'etag': '"3482549379738718"',
 'id': 'aov56m37rbb6rrdp1gq1d5n2rk',
 'status': 'confirmed',
 'htmlLink': 'https://www.google.com/calendar/event?eid=YW92NTZtMzdyYmI2cnJkcDFncTFkNW4ycmsgbmV2ZXJsaXNoQG0',
 'created': '2025-03-06T15:24:49.000Z',
 'updated': '2025-03-06T15:24:49.869Z',
 'summary': 'Lunch',
 'creator': {'email': 'neverlish@gmail.com', 'self': True},
 'organizer': {'email': 'neverlish@gmail.com', 'self': True},
 'start': {'dateTime': '2024-08-14T13:00:00+09:00', 'timeZone': 'Asia/Seoul'},
 'end': {'dateTime': '2024-08-14T14:00:00+09:00', 'timeZone': 'Asia/Seoul'},
 'iCalUID': 'aov56m37rbb6rrdp1gq1d5n2rk@google.com',
 'sequence': 0,
 'reminders': {'useDefault': True},
 'eventType': 'default'}

# End-to-End Function Call Workflow

1. Function 준비 (또는 작성)
2. Tools 정의
3. LLM API 호출
4. (필요 시) Function 실행 - 일반 발화가 필요한 경우 실행 필요 X
5. LLM API 최종 호출